In [1]:
import tensorflow as tf
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

In [2]:
## Define train and test paths
train_path = './FRE2013/train'
test_path = './FRE2013/test'

In [7]:
from datetime import datetime
from packaging import version

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten,MaxPool2D,Conv2D,Dropout,MaxPooling2D,BatchNormalization,Input
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

In [4]:
%load_ext tensorboard
import tensorboard
tensorboard.__version__

'2.9.0'

In [5]:
!rm -rf ./logs/

In [8]:
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [9]:
train_set=ImageDataGenerator(
        rescale=1. / 255,
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True,
).flow_from_directory(train_path,
                                             target_size=(48,48),
                                             batch_size=16,
                                             color_mode='grayscale',
                                             class_mode='categorical')
test_set=ImageDataGenerator().flow_from_directory(test_path,
                                             target_size=(48,48),
                                             batch_size=16,
                                             color_mode='grayscale',
                                             class_mode='categorical')

Found 21107 images belonging to 4 classes.
Found 5278 images belonging to 4 classes.


In [ ]:
## MODEL 1
model1 = Sequential()

model1.add(BatchNormalization(input_shape=(48,48,1)))
model1.add(Conv2D(32, kernel_size=(5, 5), activation='relu',input_shape=(48,48,1), padding='same'))
model1.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model1.add(BatchNormalization())

model1.add(Conv2D(32, kernel_size=(4, 4), activation='relu', padding='same'))
model1.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model1.add(BatchNormalization())

model1.add(Conv2D(64, kernel_size=(5, 5), activation='relu', padding='same'))
model1.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model1.add(BatchNormalization())

model1.add(Dropout(0.3))
model1.add(Dense(1024))
model1.add(Dense(4, activation='softmax'))

In [ ]:
# MODEL 2

model2 = Sequential()

model2.add(BatchNormalization(input_shape=(48,48,1)))
model2.add(Conv2D(32, (3, 3), activation='relu',padding='same', input_shape=(48,48,1),name="conv1"))
model2.add(BatchNormalization())
#mode2l.add(MaxPooling2D(pool_size=(2, 2),name="maxpool1"))
model2.add(Dropout(0.2))

model2.add(Conv2D(32, (3, 3), activation='relu',padding='same',name="conv2"))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2, 2),name="maxpool2"))         
model2.add(Dropout(0.2))

model2.add(Conv2D(64, (3, 3), activation='relu',padding='same',name="conv3"))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2, 2),name="maxpool3"))
model2.add(Dropout(0.25))

model2.add(Conv2D(64, (3, 3), activation='relu',padding='same',name="conv4"))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2, 2),name="maxpool4"))
model2.add(Dropout(0.5))

model2.add(Flatten())
model2.add(Dense(1024, activation='relu',name='fc1'))
model2.add(Dropout(0.3))
model2.add(BatchNormalization())

model2.add(Dense(4, activation='softmax',name='fcsoftmax'))

In [42]:
# MODEL 3

model3 = Sequential()

model3.add(Conv2D(32, (3, 3), activation='relu',padding='same', input_shape=(48,48,1),name="conv1"))
model3.add(BatchNormalization())
model3.add(Dropout(0.2))

model3.add(Conv2D(32, (3, 3), activation='relu',padding='same',name="conv2"))
model3.add(BatchNormalization())
model3.add(MaxPooling2D(pool_size=(2, 2),name="maxpool2"))     
model3.add(Dense(256, activation='relu'))    
model3.add(Dropout(0.2))

model3.add(Conv2D(64, (3, 3), activation='relu',padding='same',name="conv3"))
model3.add(BatchNormalization())
model3.add(MaxPooling2D(pool_size=(2, 2),name="maxpool3"))
model3.add(Dense(128, activation='relu'))  
model3.add(Dropout(0.1))

model3.add(Conv2D(64, (3, 3), activation='relu',padding='same',name="conv4"))
model3.add(BatchNormalization())
model3.add(MaxPooling2D(pool_size=(2, 2),name="maxpool4"))
model3.add(Dense(256, activation='relu'))  
model3.add(Dropout(0.2))

model3.add(Flatten())
model3.add(Dense(1024, activation='relu',name='fc1'))
model3.add(Dropout(0.3))
model3.add(BatchNormalization())

model3.add(Dense(1024, activation='relu',name='fc2'))
model3.add(Dropout(0.1))
model3.add(BatchNormalization())

model3.add(Dense(4, activation='softmax',name='fcsoftmax'))

In [ ]:
model2.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
model2.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_19 (Bat  (None, 48, 48, 1)        4         
 chNormalization)                                                
                                                                 
 conv1 (Conv2D)              (None, 48, 48, 32)        320       
                                                                 
 batch_normalization_20 (Bat  (None, 48, 48, 32)       128       
 chNormalization)                                                
                                                                 
 dropout_12 (Dropout)        (None, 48, 48, 32)        0         
                                                                 
 conv2 (Conv2D)              (None, 48, 48, 32)        9248      
                                                                 
 batch_normalization_21 (Bat  (None, 48, 48, 32)      

In [ ]:
history=model2.fit(train_set,epochs=25,validation_data=test_set,callbacks=[tensorboard_callback])

Epoch 1/100
1320/1320 [==============================] - 18s 12ms/step - loss: 1.4380 - accuracy: 0.4034 - val_loss: 1.1614 - val_accuracy: 0.5036
Epoch 2/100
1320/1320 [==============================] - 16s 12ms/step - loss: 1.1548 - accuracy: 0.4986 - val_loss: 1.0837 - val_accuracy: 0.5521
Epoch 3/100
1320/1320 [==============================] - 15s 11ms/step - loss: 1.1001 - accuracy: 0.5242 - val_loss: 1.0604 - val_accuracy: 0.5618
Epoch 4/100
1320/1320 [==============================] - 15s 11ms/step - loss: 1.0637 - accuracy: 0.5468 - val_loss: 0.9740 - val_accuracy: 0.5839
Epoch 5/100
1320/1320 [==============================] - 16s 12ms/step - loss: 1.0357 - accuracy: 0.5637 - val_loss: 0.9571 - val_accuracy: 0.5961
Epoch 6/100
1320/1320 [==============================] - 16s 12ms/step - loss: 1.0141 - accuracy: 0.5674 - val_loss: 0.9491 - val_accuracy: 0.6016
Epoch 7/100
1320/1320 [==============================] - 16s 12ms/step - loss: 0.9969 - accuracy: 0.5833 - val_loss: 0

In [43]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0,patience=3,verbose=1)
model3.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model3.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 48, 48, 32)        320       
                                                                 
 batch_normalization_52 (Bat  (None, 48, 48, 32)       128       
 chNormalization)                                                
                                                                 
 dropout_51 (Dropout)        (None, 48, 48, 32)        0         
                                                                 
 conv2 (Conv2D)              (None, 48, 48, 32)        9248      
                                                                 
 batch_normalization_53 (Bat  (None, 48, 48, 32)       128       
 chNormalization)                                                
                                                                 
 maxpool2 (MaxPooling2D)     (None, 24, 24, 32)      

In [44]:
history=model3.fit(train_set,epochs=100,validation_data=test_set,batch_size = 128,callbacks=[tensorboard_callback,early_stopping])

Epoch 1/100
1320/1320 [==============================] - 25s 18ms/step - loss: 1.4310 - accuracy: 0.4110 - val_loss: 1.3079 - val_accuracy: 0.4331
Epoch 2/100
1320/1320 [==============================] - 24s 18ms/step - loss: 1.0986 - accuracy: 0.5311 - val_loss: 0.9989 - val_accuracy: 0.5794
Epoch 3/100
1320/1320 [==============================] - 25s 19ms/step - loss: 1.0189 - accuracy: 0.5667 - val_loss: 1.0658 - val_accuracy: 0.5443
Epoch 4/100
1320/1320 [==============================] - 23s 18ms/step - loss: 0.9694 - accuracy: 0.5922 - val_loss: 1.0128 - val_accuracy: 0.5815
Epoch 5/100
 931/1320 [====================>.........] - ETA: 6s - loss: 0.9318 - accuracy: 0.6061

In [14]:
model3.save('model3.hdf5')

In [38]:
## Data Generator
learning_rate = 1e-3
batch_size = 128
num_classes = 4

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True,
)

train_generator =train_datagen.flow_from_directory(
    train_path,
    target_size=(48, 48),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=True,
)

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(48, 48),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=True,
)

Found 21107 images belonging to 4 classes.
Found 5278 images belonging to 4 classes.


In [39]:
## Optimized model

initializer = tf.keras.initializers.HeNormal()

model4 = Sequential()
model4.add(Input(shape=(48,48,1)))
# model4.add(Conv2D(filters=256,kernel_size=3,activation='relu',padding='same'))

model4.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same'))
model4.add(BatchNormalization())
model4.add(MaxPooling2D(pool_size=(2,2)))
model4.add(Dropout(0.4))

model4.add(Conv2D(filters=384,kernel_size=(3,3),activation='relu',padding='same'))
model4.add(BatchNormalization())
model4.add(MaxPooling2D(pool_size=(2,2)))
model4.add(Dropout(0.4))

model4.add(Conv2D(filters=192,kernel_size=(3,3),activation='relu',padding='same'))
model4.add(BatchNormalization())
model4.add(MaxPooling2D(pool_size=(2,2)))
model4.add(Dropout(0.4))

model4.add(Conv2D(filters=384,kernel_size=(3,3),activation='relu',padding='same'))
model4.add(BatchNormalization())
model4.add(MaxPooling2D(pool_size=(2,2)))
model4.add(Dropout(0.4))

model4.add(Flatten())
model4.add(Dense(256,activation='relu'))
model4.add(BatchNormalization())
model4.add(Dropout(0.3))

# model4.add(Dense(128,activation='relu'))
# model4.add(BatchNormalization())
# model4.add(Dropout(0.3))
model4.add(Dense(4,activation='relu'))

In [40]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0,patience=3,verbose=1)

In [41]:
model4.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model4.summary()
history=model4.fit(train_generator,epochs=100,validation_data=test_generator,batch_size = 128,callbacks=[tensorboard_callback,early_stopping])

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 48, 48, 512)       5120      
                                                                 
 batch_normalization_47 (Bat  (None, 48, 48, 512)      2048      
 chNormalization)                                                
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 24, 24, 512)      0         
 g2D)                                                            
                                                                 
 dropout_46 (Dropout)        (None, 24, 24, 512)       0         
                                                                 
 conv2d_31 (Conv2D)          (None, 24, 24, 384)       1769856   
                                                                 
 batch_normalization_48 (Bat  (None, 24, 24, 384)     